In [ ]:
!nvidia-smi

In [ ]:
! pip install --quiet sentence-transformers==2.2.2 
! pip install --quiet InstructorEmbedding
! pip install --quiet langchain
! pip install --quiet langchain-community
! pip install --quiet langchain-google-genai
! pip install --quiet google-generativeai<0.6.0
! pip install --quiet faiss-gpu

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "<your_api_key>"

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"], temperature=0.1)

In [ ]:
ans = llm.invoke("Write a poem on shaastra 2025")
ans.pretty_print()

In [ ]:
ans = llm.invoke("What do you know about OpenAI?")
ans.pretty_print()

In [ ]:
ans = llm.invoke("Who is the director of IITM?")
ans.pretty_print()

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=os.environ["GOOGLE_API_KEY"], temperature=1)

In [ ]:
ans = llm.invoke("Who is the director of IITM?")
ans.pretty_print()

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/kaggle/input/data-faqs/faqs.csv', source_column="prompt", encoding="Windows-1252")

# Store the loaded data in the 'data' variable
data = loader.load()

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

# Initialize instructor embeddings using the Hugging Face model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")


In [ ]:
e = instructor_embeddings.embed_query("What is your refund policy?")

In [ ]:
len(e)

In [ ]:
e[0:5]

In [ ]:
X.shape

In [ ]:
text_1 = instructor_embeddings.embed_query("What is your refund policy?")
text_2 = instructor_embeddings.embed_query("India has a very good cricket team")

In [ ]:
from scipy import spatial

result = 1 - spatial.distance.cosine(text_1, text_2)
result

In [ ]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [ ]:
!nvidia-smi

In [ ]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)

In [ ]:
chain('Do you provide job assistance and also do you provide job gurantee?')

In [ ]:
ans = llm.invoke("Do you guys provide internship and also do you offer EMI payments?")
ans.pretty_print()

In [ ]:
chain("Do you guys provide internship and also do you offer EMI payments?")

In [ ]:
chain("do you have javascript course?")